In [ ]:

import datetime

import netCDF4
import pandas as pd
import xarray as xr
import matplotlib as mpl
import cdsapi
import numpy as np
import seaborn as sns
import warnings
import geopandas as gp
import regionmask
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings('ignore')
from tabulate import tabulate
plt.style.use('default')
import cftime
import cartopy.crs as ccrs
import pvlib
import os
from global_land_mask import globe
import CMIP6_light_map
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import box, mapping
import geopandas as gpd

def convert_to_180(ds):
    return ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')

%%time
# Here, we use the ERA5 data and the Python API interface to cdsapi.
#
# https://www.ecmwf.int/sites/default/files/elibrary/2015/18490-radiation-quantities-ecmwf-model-and-mars.pdf
# ERA5, Monthly Means of Daily Forecast Accumulations
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
def get_era5(station, step):
    c = cdsapi.Client()

    years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015','2016','2017','2018','2019']

    c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
        'variable': [
            'clear_sky_direct_solar_radiation_at_surface',
            'downward_uv_radiation_at_the_surface',
            'surface_solar_radiation_downward_clear_sky',
            'surface_solar_radiation_downwards',
        ],
        'year': ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015','2016','2017','2018','2019'],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            90, -180, 50,
            180,
        ],
        'format': 'netcdf',
    },
        "../oceanography/light/ncfiles/ERA5_global_hourly_shortwave.nc")

In [ ]:
%%time
stations=[{"name":"Chukchi_sea",
           "lat":69.25,
           "lon":-171}]

for i, station in enumerate(stations):
    step=3
    get_era5(station, step)